# 2.3 方策反復法の実装

前節では迷路内をAgentがランダムに行動する方策を実装した。  
これから、Agentが一直線にゴールへ向かうように方策を学習させる。
この方法は大きく二つの方法がある。

## 方策反復法
うまくいったケースの行動を重要視する作戦

具体的な実装として、以下のような方策勾配法がある


$$
\theta_{s_i,a_j} \leftarrow \theta_{s_i,a_j} + \eta \cdot \Delta \theta_{s, a_j} \\ \Delta \theta_{s, a_j} = \{ N(s_i, a_j) - P(s_i, a_j) N (s_i, a) \}  / T 
$$

(softmax関数を使用した確率への変換とREINFORCEアルゴリズムに従うとこのような更新式が得られる)

## 価値反復法
ゴール以外の状態にも価値をつけてあげる

具体的な実装として、以下のようなQ学習がある


$$
Q(s_t, a_t) \leftarrow Q(s_t, a_t) + \eta \times (R_{t+1} + \gamma \max_a Q(s_{t+1}, a) - Q(s_t, a_t))
$$


ここでは方策反復法を実装する。
前節では単純にθを割合に変換して方策を求める関数を実装したが、本節ではθを変換する際にsoftmax関数を使用する。
ここでsoftmax関数を使用したのは、Θの更新式によってΘが負の値になっても方策を導出できるようにするため。


In [1]:
import numpy as np

def simple_convert_into_pi_from_theta(theta):
    """単純に割合を計算する"""
    [m,n] = theta.shape
    pi = np.zeros((m,n))
    for i in range(0,m):
        pi[i,:] = theta[i,:] / np.nansum(theta[i,:])
        
    pi = np.nan_to_num(pi)
    return pi
    
    
# 前節で用いた`simple_convert_into_pi_from_theta`と比較しながら読んで欲しい。
def softmax_convert_into_pi_from_theta(theta):
    """ソフトマックス関数で割合を計算する"""
    beta = 1.0
    [m,n] = theta.shape
    pi = np.zeros((m,n))
    
    exp_theta = np.exp(beta * theta)
    
    for i in range(0,m):
        pi[i,:] = exp_theta[i,:] / np.nansum(exp_theta[i,:])
        
    pi = np.nan_to_num(pi)
    return pi
    
    
theta_0 = np.array([
    [np.nan, 1, 1, np.nan],
    [np.nan, 1, np.nan, 1],
    [np.nan, np.nan, 1, 1],
    [1, 1, 1, np.nan],
    [np.nan, np.nan, 1, 1],
    [1, np.nan, np.nan, np.nan],
    [1, np.nan, np.nan, np.nan],
    [1, 1, np.nan, np.nan],
])    

pi_0 = softmax_convert_into_pi_from_theta(theta_0)
print(pi_0)

[[0.         0.5        0.5        0.        ]
 [0.         0.5        0.         0.5       ]
 [0.         0.         0.5        0.5       ]
 [0.33333333 0.33333333 0.33333333 0.        ]
 [0.         0.         0.5        0.5       ]
 [1.         0.         0.         0.        ]
 [1.         0.         0.         0.        ]
 [0.5        0.5        0.         0.        ]]


初期状態では前節と同じ結果になるが、学習に従いθの値が変わると、関数`softmax_convert_into_pi_from_theta`の計算結果は単純な割合計算とは異なる結果となる。

続いて、softmax関数による$\pi_{\theta} (s,a)$に従ってAgentを行動させる関数を定義する。
θの更新時に必要な情報を取得するため、状態だけでなくその時採用した行動も取得する。


In [ ]:
def get_action_and_next_s(pi,s):
    direction = ["up", "right", "down", "left"]

    # 二重配列piのs番目の配列の要素(確率)に従って、directionが選択される
    next_direction = np.random.choice(direction, p=pi[s, :])

    if next_direction == "up":
        action = 0
        s_next = s - 3
    elif next_direction == "right":
        action = 1
        s_next = s + 1
    elif next_direction == "down":
        action = 2
        s_next = s + 3
    else:
        action = 3
        s_next = s - 1

    return [action, s_next]


ゴールにたどり着くまでAgentを移動させ続ける関数を定義する。
Agentの状態と、その状態で採用した行動の組みの配列を返す。


In [ ]:
def goal_maze_ret_s_a(pi):
    s = 0
    s_a_history = [[0, np.nan]]
    
    while True :
        [action, next_s] = get_action_and_next_s(pi,s)
        # -1は現在の状態(=一番最後の状態)を指す
        # 必要なら板書で説明する
        s_a_history[-1][1] = action
    
        s_a_history.append([next_s, np.nan])
        
        # ゴールなら終了
        if next_s == 8:
            break
        else:
            s = next_s
            
    return s_a_history


s_a_history = goal_maze_ret_s_a(pi_0)
print(s_a_history)
